In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Road traffic video for object recognition.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\Traffic Flow In The Highway - 4K Stock Videos _ NoCopyright _ AllVideoFree.mp4',
 'video_footages\\vecteezy_busy-street-in-evening-hong-kong_28840705 (1) (1).mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4']

In [6]:
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)

lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

color = (0, 255, 0)

cap = cv2.VideoCapture(video_files[1])
ret, old_frame = cap.read()
height, width, layers = old_frame.shape
new_h = height/2
new_w = width/2
old_frame = cv2.resize(old_frame, (int(new_w), int(new_h)))
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_frame)




while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.resize(frame, (int(new_w), int(new_h)))
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        next, status, error = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, prev, None, **lk_params)

        good_old = prev[status == 1]

        good_new = next[status == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
        # Returns a contiguous flattened array as (x, y) coordinates for new point
            a, b = new.ravel()
        # Returns a contiguous flattened array as (x, y) coordinates for old point
            mask = cv2.rectangle(frame, (int(a-5), int(b-5)), (int(a+5),int(b+5)), color, thickness=2)

        output = cv2.add(frame, mask)

        cv2.imshow("sparse optical flow", output)

        prev = good_new.reshape(-1, 1, 2)

        old_gray = frame_gray.copy()

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
        
    else:
        break


cap.release()

cv2.destroyAllWindows()